# Bahdanau 注意力

`sec_seq2seq`中探讨了机器翻译问题：
通过设计一个基于两个循环神经网络的编码器-解码器架构，
用于序列到序列学习。
具体来说，循环神经网络编码器将长度可变的序列转换为固定形状的上下文变量，
然后循环神经网络解码器根据生成的词元和上下文变量按词元生成输出（目标）序列词元。

**然而，即使并非所有输入（源）词元都对解码某个词元都有用，
在每个解码步骤中仍使用编码 *相同* 的上下文变量。
有什么方法能改变上下文变量呢？**

我们试着从`Graves.2013`中找到灵感：
在为给定文本序列生成手写的挑战中，
Graves设计了一种可微注意力模型，
将文本字符与更长的笔迹对齐，
其中对齐方式仅向一个方向移动。
受学习对齐想法的启发，Bahdanau等人提出了一个没有严格单向对齐限制的可微注意力模型`Bahdanau.Cho.Bengio.2014`。
在预测词元时，如果不是所有输入词元都相关，模型将仅对齐（或参与）输入序列中与当前预测相关的部分。这是通过将上下文变量视为注意力集中的输出来实现的。

## 动机

### seq2seq缺陷

Encoder 一方面以 Decoder 输出 RNN 的 hidden state 作为 Decoder 的 RNN 初始化 hidden state；另一方面，Encoder 只把一个句子中的最后一个时刻（word）的最后一层的隐藏状态当做 context（上下文），与 Decoder 的 Input 之一 Embedding 进行拼接，作为 Decoder 的另一个输入。

**但是最后一个时刻（word）的最后一层这一 context，尽管包含了之前的 time_step 的信息，但也是间接的信息，且失去了位置信息。**

把最后一个时刻（word）的最后一层作为 context，是不合适的，因为从道理上讲，Encoder（源语言）和 Decoder（目标翻译语言）的位置最好一一对应，而不是统一拿 Encoder 的 RNN 的最后一个 time_step 的最后一层当做 Decoder 的 RNN 的输入的一部分。举个例子，在 Decoder 第一个位置预测 bonjour 的时候，不应该拿 Encoder 的最后一个位置的句号的输出作为 context，而是应该拿第一个位置的 hello 的输出作为 context。

### attention改进

seq2seq 加入 Attention机制，则允许 seq2seq 修正这一缺陷。具体做法：$\mathbf{c}_{t'} = \sum_{t=1}^T \alpha(\mathbf{s}_{t' - 1}, \mathbf{h}_t) \mathbf{h}_t$。

其中，下标 $t$ 为 Encoder 的下标，下标 $t'$ 为Decoder的下标；
$h_t$ 为 Encoder 的 RNN 的 hidden state 输出，既是 key 又是 value；
$s_{t'-1}$ 为 $t‘-1$ 时刻 Decoder 的 hidden state输出；$c_t'$ 为Decoder 在 $t'$ 时刻的上下文（context）变量；$s$，$h$ 和 $c$ 都是向量；$\alpha$ 函数是注意力权重函数。相当于原版的 seq2seq 的上下文变量 $c$，在任何解码时间步 $t'$ 都会被 $c_t'$ 替换，更加灵活。

具体来说，假设 Encoder 的 RNN 中，一个英文句子的长度（time_step）为 3，则有 3 对 key value pair（键值对），其中 key=value，第 $i$ 个 key 或 value 就是第 $i$ 个 word 的 RNN 的 hidden state 这一输出。把它们放入Attention中，此时 $h_t$ = 第 $t$ 个 key 或value。

与此同时，在 Decoder 的 RNN 中，对上一个词的 hidden state 输出就是 query，即 $s_{t'-1}$ = query。之所以在 Decoder 使用 RNN 的输出，而不是 input 即 Embedding，是因为这样可以与 Encoder 的 RNN 的输出处于同一个语义空间，方便用 query 对 key value pair 进行查询。Attention 的结果 $f(x)$ 即 $c_t'$，就能替代之前的 context（Encoder 最后一个时刻的最后一层的输出），与下一个词的 Embeding 进行 concat，作为 Decoder 的输入。

更加形象的解释：
原始的 seq2seq 只把 Encoder 的 RNN 最后一层的最后一个单词的输出拿出来作为 $c$，而加入 Attention 的 seq2seq 则把 Encoder 的 RNN 的最后一层的所有单词（key value pair）都拿出来，做一个加权平均处理，同时根据位置（Decoder 的 $t'$）的不同，一开始对 Encoder 中 $t$ 偏小的词赋予更多权重，越往后则对 Encoder 中 $t$ 偏大的词赋予更多权重。

## 小tips

自己从零实现的 gru、lstm、rnn 和 pytorch 实现的不一样，在 pytorch 版本中是没有实现最后的输出层的，目的是为了让我们自定义输出层，跟我们从零实现的不一样。
所以：

- output 形状为 **(num_steps，batch_size，num_hiddens)**，即每个时间步中最后一层的隐藏状态。（这里是指 Encoder 输出的结构）

- state 形状为 **(num_layers, batch_size, num_hiddens)**，即最后一个时间步所有层的隐藏状态。

## 模型

下面描述的Bahdanau注意力模型将遵循`sec_seq2seq`中的相同符号表达。
这个新的基于注意力的模型与`sec_seq2seq`中的模型相同，
只不过`eq_seq2seq_s_t`中的上下文变量 $\mathbf{c}$ 在任何解码时间步 $t'$ 都会被 $\mathbf{c}_{t'}$ 替换。
假设输入序列中有 $T$ 个词元，
解码时间步 $t'$ 的上下文变量是注意力集中的输出：

$$\mathbf{c}_{t'} = \sum_{t=1}^T \alpha(\mathbf{s}_{t' - 1}, \mathbf{h}_t) \mathbf{h}_t$$

其中，时间步 $t' - 1$ 时的解码器隐状态 $\mathbf{s}_{t' - 1}$ 是查询，
编码器隐状态 $\mathbf{h}_t$ 既是键，也是值，
注意力权重 $\alpha$ 是使用`eq_attn-scoring-alpha`所定义的加性注意力打分函数计算的。

与`fig_seq2seq_details`中的循环神经网络编码器-解码器架构略有不同，
`fig_s2s_attention_details`描述了Bahdanau注意力的架构。

![一个带有Bahdanau注意力的循环神经网络编码器-解码器模型](../img/seq2seq-attention-details.svg)

In [3]:
import torch
from torch import nn
from d2l import torch as d2l

## 定义注意力解码器

下面看看如何定义Bahdanau注意力，实现循环神经网络编码器-解码器。
其实，我们只需重新定义解码器即可。
为了更方便地显示学习的注意力权重，
以下`AttentionDecoder`类定义了带有注意力机制解码器的基本接口。

In [4]:
#@save
class AttentionDecoder(d2l.Decoder):
    """带有注意力机制解码器的基本接口"""
    def __init__(self, **kwargs):
        super(AttentionDecoder, self).__init__(**kwargs)

    @property
    def attention_weights(self):
        raise NotImplementedError

接下来，让我们在接下来的`Seq2SeqAttentionDecoder`类中实现带有Bahdanau注意力的循环神经网络解码器。
首先，初始化解码器的状态，需要下面的输入：

1. 编码器在所有时间步的最终层隐状态，将作为注意力的键和值。
2. 上一时间步的编码器全层隐状态，将作为初始化解码器的隐状态。
3. 编码器有效长度（排除在注意力池中填充词元）。

在每个解码时间步骤中，解码器上一个时间步的最终层隐状态将用作查询。
因此，注意力输出和输入嵌入都连结为循环神经网络解码器的输入。

**`Seq2SeqAttentionDecoder`类写的非常清楚，值得认真阅读每一行。**

其核心部分用到了`sec_attention-scoring-functions`中的`AdditiveAttention`类，应及时回顾类的定义、参数含义及其用法，否则会严重影响理解的速度。

In [5]:
class Seq2SeqAttentionDecoder(AttentionDecoder):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, dropout=0, **kwargs):
        super(Seq2SeqAttentionDecoder, self).__init__(**kwargs)
        
        # 加性注意力机制，前三个参数分别为 key_size, query_size, num_hiddens
        self.attention = d2l.AdditiveAttention(num_hiddens, num_hiddens, num_hiddens, dropout)
        # 词嵌入层
        self.embedding = nn.Embedding(vocab_size, embed_size)
        # 输入是词嵌入和加性注意力的输出，输出是词嵌入
        self.rnn = nn.GRU(embed_size + num_hiddens, num_hiddens, num_layers, dropout=dropout)
        # 全连接层，将 RNN 的输出映射到词汇表的维度
        self.dense = nn.Linear(num_hiddens, vocab_size)

    def init_state(self, enc_outputs, enc_valid_lens, *args):
        # outputs 形状为 (num_steps，batch_size，num_hiddens)
        # hidden_state 形状为 (num_layers, batch_size, num_hiddens)
        outputs, hidden_state = enc_outputs
        # 返回的（经过 permutate 后）outputs 形状为 (batch_size，num_steps，num_hiddens)
        return (outputs.permute(1, 0, 2), hidden_state, enc_valid_lens)

    def forward(self, X, state):
        # enc_outputs 形状为 (batch_size, num_steps, num_hiddens)
        # hidden_state 形状为 (num_layers, batch_size, num_hiddens)
        enc_outputs, hidden_state, enc_valid_lens = state
        # 输出（变换后） X 形状为 (num_steps, batch_size, embed_size)
        X = self.embedding(X).permute(1, 0, 2)
        outputs, self._attention_weights = [], []

        # 对于每一步 x，需要结合的上下文信息 context 是不同的
        # 此时 X 形状为 (num_steps, batch_size, embed_size)
        for x in X:
            # 取解码器的最后一个隐藏状态 hidden_state[-1] 并 unsqueeze 后作为 query
            # query 形状为 (batch_size, 1, num_hiddens)；1 代表只有 1 个查询，num_hiddens 代表 query_size
            query = torch.unsqueeze(hidden_state[-1], dim=1)

            # keys values 相同，都为 enc_outputs
            # enc_outputs 形状为 (batch_size, num_steps, num_hiddens)；num_steps 代表 “键－值”对的个数，num_hiddens 代表 key/value_size
            # enc_valid_lens 形状为 (batch_size,) 的向量，元素大小代表有效“键－值”对的个数（即剔除 padding 的 step）
            context = self.attention(query, enc_outputs, enc_outputs, enc_valid_lens)
            # 输出 context 形状为 (batch_size, 1, num_hiddens)
            # 解读：num_queries 为 1；value_size 为 num_hiddens

            # 在特征维度上连结
            x = torch.cat((context, torch.unsqueeze(x, dim=1)), dim=-1)
            # 将 x 变形为 (1, batch_size, embed_size + num_hiddens)
            out, hidden_state = self.rnn(x.permute(1, 0, 2), hidden_state)
            # outputs 形状为 (num_steps, batch_size, num_hiddens)
            outputs.append(out)
            self._attention_weights.append(self.attention.attention_weights)

        # 全连接层变换后形状为 (num_steps, batch_size, vocab_size)
        outputs = self.dense(torch.cat(outputs, dim=0))

        # (经 permute) 返回的 outputs 形状为 (batch_size, num_steps, vocab_size)
        return outputs.permute(1, 0, 2), [enc_outputs, hidden_state, enc_valid_lens]

    @property
    def attention_weights(self):
        return self._attention_weights

接下来，使用包含 7 个时间步（num_steps）的 4 个序列输入（batch_size）的小批量测试 Bahdanau 注意力解码器。

In [10]:
encoder = d2l.Seq2SeqEncoder(vocab_size=10, embed_size=8, num_hiddens=16, num_layers=2)
encoder.eval()

decoder = Seq2SeqAttentionDecoder(vocab_size=10, embed_size=8, num_hiddens=16, num_layers=2)
decoder.eval()

# X 形状为 (batch_size, num_steps)
X = torch.zeros((4, 7), dtype=torch.long)
# 初始化 decoder 的状态
state = decoder.init_state(encoder(X), None)
output, state = decoder(X, state)

# output: (batch_size, num_steps, vocab_size); state: [enc_outputs, hidden_state, enc_valid_lens]
print(output.shape, len(state))
# state[0]: encoder_outputs 形状为 (batch_size, num_steps, num_hiddens)
# state[1]: hidden_state 形状为 (num_layers, batch_size, num_hiddens)
print(state[0].shape, len(state[1]), state[1].shape)

torch.Size([4, 7, 10]) 3
torch.Size([4, 7, 16]) 2 torch.Size([2, 4, 16])


## 训练

与`sec_seq2seq_training`类似，
我们在这里指定超参数，实例化一个带有Bahdanau注意力的编码器和解码器，
并对这个模型进行机器翻译训练。
由于新增的注意力机制，训练要比没有注意力机制的`sec_seq2seq_training`慢得多。

In [11]:
embed_size, num_hiddens, num_layers, dropout = 32, 32, 2, 0.1
batch_size, num_steps = 64, 10
lr, num_epochs, device = 0.005, 250, d2l.try_gpu()

train_iter, src_vocab, tgt_vocab = d2l.load_data_nmt(batch_size, num_steps)

encoder = d2l.Seq2SeqEncoder(len(src_vocab), embed_size, num_hiddens, num_layers, dropout)
decoder = Seq2SeqAttentionDecoder(len(tgt_vocab), embed_size, num_hiddens, num_layers, dropout)
net = d2l.EncoderDecoder(encoder, decoder)

d2l.train_seq2seq(net, train_iter, lr, num_epochs, tgt_vocab, device)

loss 0.021, 14579.6 tokens/sec on cpu


模型训练后，我们用它将几个英语句子翻译成法语并计算它们的 BLEU 分数。

如果对预测函数有些遗忘，可以回到`sec_seq2seq`进行温习。

In [12]:
engs = ['go .', "i lost .", 'he\'s calm .', 'i\'m home .']
fras = ['va !', 'j\'ai perdu .', 'il est calme .', 'je suis chez moi .']

for eng, fra in zip(engs, fras):
    translation, dec_attention_weight_seq = d2l.predict_seq2seq(net, eng, src_vocab, tgt_vocab, num_steps, device, True)
    print(f'{eng} => {translation}, ',
          f'bleu {d2l.bleu(translation, fra, k=2):.3f}')

go . => va !,  bleu 1.000
i lost . => j'ai perdu .,  bleu 1.000
he's calm . => il est riche .,  bleu 0.658
i'm home . => je suis chez moi .,  bleu 1.000


In [21]:
# 拼接解码器在每个时间步（step）中的注意力权重
# num_steps（解码器的步数）等同于 key_value_pairs（键值对的数量）
attention_weights = torch.cat([step[0][0][0] for step in dec_attention_weight_seq], 0).reshape((1, 1, -1, num_steps))

训练结束后，下面通过可视化注意力权重会发现，
**每个查询都会在键值对上分配不同的权重，这说明在每个解码步中，输入序列的不同部分被选择性地聚集在注意力池中。**

**纵坐标代表 query，横坐标代表 key，颜色深浅代表权重值大小。**

In [22]:
# 最后一次的输入序列为 'i'm home .'，输出序列为 'je suis chez moi .'

# 输入序列长度为 1 + 3 = 4（包含序列开始词元 <bos>）；故键的个数为 4
# 输出序列长度为 5 + 1 = 6（包含序列结束词元 <eos>）；故查询个数为 6

# show_heatmaps 的输入形状为（要显示的行数，要显示的列数，查询的数目，键的数目）
d2l.show_heatmaps(
    attention_weights[:, :, :, :len(engs[-1].split()) + 1].cpu(),
    xlabel='Key positions', ylabel='Query positions')

## 小结

* 在预测词元时，如果不是所有输入词元都是相关的，那么具有Bahdanau注意力的循环神经网络编码器-解码器会有选择地统计输入序列的不同部分。这是通过将上下文变量视为加性注意力池化的输出来实现的。
* 在循环神经网络编码器-解码器中，Bahdanau注意力将上一时间步的解码器隐状态视为查询，在所有时间步的编码器隐状态同时视为键和值。

## 练习

1. 在实验中用LSTM替换GRU。
2. 修改实验以将加性注意力打分函数替换为缩放点积注意力，它如何影响训练效率？

[Discussions](https://discuss.d2l.ai/t/5754)
